In [ ]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder

In [ ]:
traindata = '/kaggle/input/limit-orderbook-data/Train_Dst_NoAuction_DecPre_CF_7.txt'
cvdata = '/kaggle/input/limit-orderbook-data/Test_Dst_NoAuction_DecPre_CF_9.txt'

In [ ]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
BATCH = 128

"""
    DataLoader file fot the LOB dataset gives out the lob matrix of size (batch_size, 1, Seq_len, 4*levels)
    Data can is taken from FI-2010 Dataset with default horizon prediction level = 50 (147, columns)
    Horizon = 10 (Level = 144) 
    For custom Lobster Data set we we change the horizon according to the dataset that we have defined  
"""


class GenMatData(Dataset):
    def __init__(self, path, level=10, horizon=147, sequencelen=100):
        metadata = np.loadtxt(path).T
        self.levels = level
        self.sequence_len = sequencelen
        self.train_data = metadata[:, :4*level]
        self.predictions = metadata[:, horizon]

    def __len__(self):
        return self.train_data.shape[0] - self.sequence_len + 1

    def __getitem__(self, item):
        lob_instance = torch.tensor(self.train_data[item:item+self.sequence_len, :]).float()
        out = torch.tensor(self.predictions[item+self.sequence_len-1])
        return lob_instance.unsqueeze(0), out-1


def LobDataLoader(batch, lobpath, shuf, level=10, seq_len=100, Horizon=147):
    dataset = GenMatData(path=lobpath, level=level, horizon=Horizon, sequencelen=seq_len)
    in_loader = DataLoader(dataset, shuffle=shuf, batch_size=batch, pin_memory=True)
    return in_loader


In [ ]:
# train_loader = LobDataLoader(batch=BATCH, lobpath=traindata, shuf=True)
# cv_loader = LobDataLoader(batch=BATCH, lobpath=cvdata, shuf=True)

In [ ]:
import torch
import torch.nn as nn

class BaseCNN(nn.Module):
    def __init__(self, in_chanel, out_chanel, kernel, strides):
        super(BaseCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_chanel, out_chanel, kernel, bias=False, stride=strides)
        self.norm1 = nn.BatchNorm2d(out_chanel)
        self.norm2 = nn.BatchNorm2d(out_chanel)
        self.norm3 = nn.BatchNorm2d(out_chanel)
        self.conv2 = nn.Conv2d(out_chanel, out_chanel, kernel_size=(3, 1), padding='same')
        self.conv3 = nn.Conv2d(out_chanel, out_chanel, kernel_size=(3, 1), padding='same')
        self.l_relu = nn.LeakyReLU(negative_slope=3e-2, inplace=True)

    def forward(self, mat):
        out = self.l_relu(self.norm1(self.conv1(mat)))
        out = self.l_relu(self.norm2(self.conv2(out)))
        out = self.l_relu(self.norm3(self.conv3(out)))
        return out


class DynamicConv(nn.Module):
    def __init__(self, in_chanel, out_chanel, dims):
        super(DynamicConv, self).__init__()
        """
                We can initialize the linear layer of the model in one other way by using 
                nn.adaptiveavgpool2d from nn and thus compute the attention in that manner 
                reducing the model weight. 
                in_chanel : input chanel in dynamic conv layer 
                out_chanel : number of output chanel 
                dim : dimension (L*W) of the input matrix 
        """
        self.dim = dims
        self.dconv1 = nn.Conv2d(in_chanel, out_chanel, kernel_size=(1, self.dim[1]), padding='same', bias=False)
        self.dconv2 = nn.Conv2d(in_chanel, out_chanel, kernel_size=(5, self.dim[1]), padding='same', bias=False)
        self.dconv3 = nn.Conv2d(in_chanel, out_chanel, kernel_size=(3, self.dim[1]), padding='same', bias=False)
        self.linear = nn.Linear(self.dim[0]*self.dim[1]*in_chanel, 3)
        self.leaky = nn.LeakyReLU(negative_slope=1e-2, inplace=True)
        self.bnorm1 = nn.BatchNorm2d(out_chanel)
        self.bnorm2 = nn.BatchNorm2d(out_chanel)
        self.bnorm3 = nn.BatchNorm2d(out_chanel)
        self.soft = nn.Softmax(dim=1)

    def forward(self, mat):
        weight = self.soft(self.linear(mat.view((mat.size(0), -1))))
        out1 = self.leaky(self.bnorm1(self.dconv1(mat))).unsqueeze(1)
        out2 = self.leaky(self.bnorm2(self.dconv2(mat))).unsqueeze(1)
        out3 = self.leaky(self.bnorm3(self.dconv3(mat))).unsqueeze(1)
        out = torch.cat((out1, out2, out3), dim=1)
        out = torch.sum(torch.mul(out, weight.reshape(weight.size(0), weight.size(1), 1, 1, 1)), dim=1)

        return out


class DeepLob(nn.Module):
    def __init__(self, in_chanels, hidden_size, num_layers, seq_length):
        super(DeepLob, self).__init__()
        self.one = BaseCNN(in_chanel=in_chanels, out_chanel=32, kernel=(1, 2), strides=(1, 2))
        self.two = BaseCNN(in_chanel=32, out_chanel=64, kernel=(1, 2), strides=(1, 2))
        self.three = BaseCNN(in_chanel=64, out_chanel=128, kernel=(1, 10), strides=(1, 10))
        self.dyconv = DynamicConv(in_chanel=128, out_chanel=128, dims=(100, 1))
        self.lstm = nn.LSTM(input_size=128, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.liner = nn.Sequential(nn.Linear(seq_length*hidden_size, hidden_size), nn.ReLU(), # Just giving last LSTM node to Dense UNIT
                                   nn.Linear(hidden_size, 3))

    def forward(self, mat):
        out_lob = self.one(mat)
        out_lob = self.two(out_lob)
        out_lob = self.three(out_lob)
        out_lob = self.dyconv(out_lob)
        out_lob = out_lob.squeeze(-1)
        out_lob = torch.transpose(out_lob, 1, 2)
        out_lob, (ho, co) = self.lstm(out_lob)
        out_lob = out_lob.reshape(out_lob.size(0), -1)
        return self.liner(out_lob)

In [ ]:
from sklearn.metrics import classification_report
def accu(model, loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    accuracy = []
    model.eval()
    with torch.no_grad():
        for mat, label in loader:
            mat = mat.to(device)
            label = label.to(device)
            pre = model(mat)
            _, pre = torch.max(pre, dim=1)
            accuracy.append((pre==label).sum()/label.size(0))
        accuracy = torch.tensor(accuracy).mean()
        return accuracy.item()

In [ ]:
import torch.optim as optim
from tqdm import trange, tqdm
def train():
    EPOCHS = 20
    traindata = '/kaggle/input/limit-orderbook-data/Train_Dst_NoAuction_DecPre_CF_7.txt'
    cvdata = '/kaggle/input/limit-orderbook-data/Test_Dst_NoAuction_DecPre_CF_9.txt'
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    loader = LobDataLoader(batch=256, lobpath=traindata, shuf=True)
    cvloader = LobDataLoader(batch=256, lobpath=cvdata, shuf=True)
    model = DeepLob(in_chanels=1, hidden_size=128, num_layers=2, seq_length=100)
    model = model.to(device)
    opt = optim.Adam(params=model.parameters(), lr=2e-3)
    crit = nn.CrossEntropyLoss()
    for epoch in range(EPOCHS):
        print('EPOCH {}'.format(epoch))
        for data, labels in tqdm(loader):
            data = data.to(device)
            labels = labels.to(device)
            prediction = model(data)
            loss = crit(prediction, labels.long())
            opt.zero_grad()
            loss.backward()
            opt.step()

        print('Epoch {} training Accuracy {} CV accuracy {}'.format(epoch, accu(model, loader), accu(model, cvloader)))
        model.train()            


In [ ]:
train()

76 accuracy Model : 
with Dyconv 2 = kernel = 5 


77.8 accu Model only taking last output from LSTM layer 

In [ ]:
# lloader = LobDataLoader(batch=32, lobpath=traindata, shuf=True)
# for data, lablel in lloader:
#     print(data.shape)
#     print(label.shape)
#     break